In [1]:
import fasttext
import numpy as np, pandas as pd


In [5]:
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')

In [11]:
dataset = pd.read_csv('train.csv')[['description', 'is_bad']].rename(columns = {'description': 'questions', 'is_bad': 'category'})
ds = pd.read_csv('val.csv')[['description', 'is_bad']].rename(columns = {'description': 'questions', 'is_bad': 'category'})

In [12]:
dataset['category'] = dataset['category'].astype(str)
ds['category'] = ds['category'].astype(str)

In [13]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [14]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /home/noname/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/noname/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/noname/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/noname/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [17]:
ds.iloc[0,0]

'Звонить 89425546881'

In [20]:
# model.predict(dataset.iloc[984482, 0])

In [21]:
# NLP Preprocess
dataset.iloc[:, 0] = dataset.iloc[:, 0].apply(lambda x: ' '.join(word_tokenize(x)))
ds.iloc[:, 0] = ds.iloc[:, 0].apply(lambda x: ' '.join(word_tokenize(x)))

# Prefixing each row of the category column with '__label__'
dataset.iloc[:, 1] = dataset.iloc[:, 1].apply(lambda x: '__label__' + x)
ds.iloc[:, 1] = ds.iloc[:, 1].apply(lambda x: '__label__' + x)

In [22]:
dataset.iloc[:, 0][0]

'Продаем диван-кровать . Удобный механизм - еврокнижка . Размер спального места 640-200 . Все подушки и подлокотники съёмные и мобильные . Ящик для белья . Продаем в связи с переездом . Муж может помочь с погрузкой . Небольшой торг возможен . Звоните в любое время .'

In [23]:
import csv

In [40]:
dataset[['category', 'questions']].to_csv('train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

ds[['category', 'questions']].to_csv('test.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")


# Training the fastText classifier
model = fasttext.train_supervised('train.txt', epoch=10, wordNgrams = 5)

# Evaluating performance on the entire test file
model.test('test.txt')                      

# Predicting on a single input
model.predict(ds.iloc[2, 0])

# Save the trained model
#model.save_model('model.bin')

(('__label__0',), array([0.97590172]))

In [26]:
model.predict(ds.iloc[2, 0])

(('__label__0',), array([0.94093859]))

In [27]:
val['description'][3]

'Продаю кровать-трансформер производства "Столплит"./\nОтличное решение для небольших детских комнат, позволяет получить большое спальное место и такой же большой стол./\nЦвет - Сосна Авола Шампань./\nВ комплекте: фирменная дополнительная секция с полками (СБ-1094, стоит наверху, может быть подвешена к стене), матрас, две лампы (одна для "настольного" режима, другая для "кроватного"), дополнительные полки в кроватном режиме, устройство для хранения БП ноутбука./\nГазовые подъёмники заменены год назад (500 кг вместо 80 кг) для облегчения подъёма кровати./\n/\nВ сумме такая комбинация (кровать, матрас и полка) на сайте stolplit точка ru стоит более 40 тысяч./\n/\nРазумный торг.'

In [41]:
predictions=[]
for line in val['description']:
    line = line.replace("\n", " ")
    
    pred_label=model.predict(line, k=-1, threshold=0.5)
    
     
# you add the list to the dataframe, then save the datframe to new csv
    if pred_label[0][0] == '__label__1':
        pred = pred_label[1][0]
    elif pred_label[0][0] == '__label__0':
        pred = 1 - pred_label[1][0]
    predictions.append(pred)
val['predict']=predictions
# df.to_csv('csv_file_w_pred.csv',sep=';',index=False)

In [42]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

roc_auc_score(val['is_bad'], val['predict'])


0.8564416803084888